In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [35]:
train_data=pd.read_csv('InsNova_data_2023_train.csv/InsNova_data_2023_train.csv')
train_data=train_data.set_index('id')
train_data.head(5)

,veh_value,exposure,veh_body,veh_age,gender,area,agecat,engine_type,max_power,driving_history_score,...,marital_status,e_bill,time_of_week_driven,time_driven,trm_len,credit_score,high_education_ind,clm,numclaims,claimcst0
id,,,,,,,,,,,,,,,,,,,,,
1,0.77,0.444504,SEDAN,4,M,D,3,petrol,147,67.0,...,S,1,weekday,6pm - 12am,6,640.448137,1.0,0,0,0.0
2,4.45,0.562183,STNWG,1,M,A,3,petrol,158,76.0,...,S,1,weekday,6am - 12pm,12,683.749691,0.0,0,0,0.0
3,4.90,0.465244,STNWG,1,F,A,3,petrol,159,58.0,...,M,1,weekday,6pm - 12am,6,653.656117,1.0,0,0,0.0
4,0.48,0.271039,PANVN,4,M,A,4,petrol,80,72.0,...,S,1,weekday,12pm - 6pm,12,642.574671,0.0,0,0,0.0
5,0.85,0.141624,SEDAN,4,F,A,5,petrol,126,91.0,...,S,0,weekday,6am - 12pm,6,647.175035,0.0,0,0,0.0


In [36]:
test_data=pd.read_csv('InsNova_data_2023_vh.csv/InsNova_data_2023_vh.csv')
test_data=test_data.set_index('id')
test_data.head(5)

,veh_value,exposure,veh_body,veh_age,gender,area,agecat,engine_type,max_power,driving_history_score,veh_color,marital_status,e_bill,time_of_week_driven,time_driven,trm_len,credit_score,high_education_ind
id,,,,,,,,,,,,,,,,,,
1,3.40,0.076279,STNWG,2,M,B,4,petrol,174,83,black,S,1,weekday,6pm - 12am,6,648.247594,0
2,2.55,0.093443,STNWG,2,F,A,3,petrol,181,65,yellow,M,0,weekday,12am - 6 am,12,637.752677,0
3,3.04,0.157762,STNWG,2,F,E,4,petrol,136,64,white,S,1,weekday,12pm - 6pm,12,661.483786,0
4,2.05,0.560735,MIBUS,4,M,C,6,dissel,164,82,gray,M,1,weekday,6am - 12pm,12,647.846365,0
5,1.93,0.258275,HBACK,2,M,C,4,dissel,89,48,black,S,0,weekday,6am - 12pm,12,640.257550,0


In [37]:
train_data.dropna(inplace=True)
train_data=train_data.drop(['numclaims','claimcst0'],axis=1)

In [23]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22619 entries, 1 to 22619
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   veh_value              22619 non-null  float64
 1   exposure               22619 non-null  float64
 2   veh_body               22619 non-null  object 
 3   veh_age                22619 non-null  int64  
 4   gender                 22619 non-null  object 
 5   area                   22619 non-null  object 
 6   agecat                 22619 non-null  int64  
 7   engine_type            22619 non-null  object 
 8   max_power              22619 non-null  int64  
 9   driving_history_score  22619 non-null  float64
 10  veh_color              22619 non-null  object 
 11  marital_status         22619 non-null  object 
 12  e_bill                 22619 non-null  int64  
 13  time_of_week_driven    22619 non-null  object 
 14  time_driven            22619 non-null  object 
 15  trm_len

In [24]:
label_mapping_body = {'BUS':0, 'CONVT':1, 'COUPE':2,'HBACK':3,'HDTOP':4,'MCARA':5, 'MIBUS':6,'PANVN':7, 'RDSTR':8, 'SEDAN':9,'STNWG':10, 'TRUCK':11,'UTE':12}
label_mapping_gender={'M':1, 'F':2}
label_mapping_area={'A':1, 'B':2,'C':3,'D':4,'E':5,'F':6}
label_mapping_engine={'petrol':1, 'electric':2, 'dissel':3, 'hybrid':4}
label_mapping_color={'black':1, 'white':2, 'gray':3, 'blue':4, 'brown':5, 'silver':6,'green':7, 'red':8, 'yellow':9}
label_mapping_status={'S':0, 'M':1}
label_mapping_week={'weekday':0,'weekend':1}
label_mapping_time={'6pm - 12am':0, '6am - 12pm':1, '12pm - 6pm':2, '12am - 6 am':3}


In [25]:
train_data['veh_body_label'] = train_data['veh_body'].map(label_mapping_body)
train_data['gender_label'] = train_data['gender'].map(label_mapping_gender)
train_data['area_label'] = train_data['area'].map(label_mapping_area)
train_data['engine_type_label'] = train_data['engine_type'].map(label_mapping_engine)
train_data['veh_color_label'] = train_data['veh_color'].map(label_mapping_color)
train_data['status_label'] = train_data['marital_status'].map(label_mapping_status)
train_data['week_label'] = train_data['time_of_week_driven'].map(label_mapping_week)
train_data['time_label'] = train_data['time_driven'].map(label_mapping_time)

test_data['veh_body_label'] = test_data['veh_body'].map(label_mapping_body)
test_data['gender_label'] = test_data['gender'].map(label_mapping_gender)
test_data['area_label'] = test_data['area'].map(label_mapping_area)
test_data['engine_type_label'] = test_data['engine_type'].map(label_mapping_engine)
test_data['veh_color_label'] = test_data['veh_color'].map(label_mapping_color)
test_data['status_label'] = test_data['marital_status'].map(label_mapping_status)
test_data['week_label'] = test_data['time_of_week_driven'].map(label_mapping_week)
test_data['time_label'] = test_data['time_driven'].map(label_mapping_time)


In [38]:
Drop_column=['veh_body','gender','engine_type','area','veh_color','marital_status','time_of_week_driven','time_driven']
train_data1=train_data.drop(columns=Drop_column,axis=1)
X_train=train_data1.drop('clm',axis=1)
y_train=train_data['clm']
X_test_vh=test_data.drop(columns=Drop_column,axis=1)
X_train

,veh_value,exposure,veh_age,agecat,max_power,driving_history_score,e_bill,trm_len,credit_score,high_education_ind
id,,,,,,,,,,
1,0.77,0.444504,4,3,147,67.0,1,6,640.448137,1.0
2,4.45,0.562183,1,3,158,76.0,1,12,683.749691,0.0
3,4.90,0.465244,1,3,159,58.0,1,6,653.656117,1.0
4,0.48,0.271039,4,4,80,72.0,1,12,642.574671,0.0
5,0.85,0.141624,4,5,126,91.0,0,6,647.175035,0.0
...,...,...,...,...,...,...,...,...,...,...
22615,3.71,0.580943,2,2,154,92.0,1,12,654.451512,0.0
22616,0.77,0.636641,4,6,160,74.0,1,12,641.163999,0.0
22617,1.95,0.709227,1,6,146,58.0,1,12,649.644433,0.0


In [39]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [40]:
rows_with_nan = X_train[X_train.isna().any(axis=1)]
print(rows_with_nan)

Empty DataFrame
Columns: [veh_value, exposure, veh_age, agecat, max_power, driving_history_score, e_bill, trm_len, credit_score, high_education_ind]
Index: []


In [41]:
from sklearn.preprocessing import StandardScaler
#Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LogisticRegression(max_iter=1000)  # Increase the number of iterations
model.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=1000)

In [42]:
# Make predictions on the test data
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Print the results
print("Accuracy:", accuracy)
print("Confusion Matrix:\n", confusion)
print("Classification Report:\n", report)

Accuracy: 0.9299292661361627
Confusion Matrix:
 [[4207    0]
 [ 317    0]]
Classification Report:
               precision    recall  f1-score   support

           0       0.93      1.00      0.96      4207
           1       0.00      0.00      0.00       317

    accuracy                           0.93      4524
   macro avg       0.46      0.50      0.48      4524
weighted avg       0.86      0.93      0.90      4524



C:\Users\Hei Yee Lau\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
C:\Users\Hei Yee Lau\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Hei Yee Lau\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Hei Yee Lau\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in label

In [43]:
#test with the test data
X_test.loc[:, 'clm'] = y_pred
X_test.to_csv('result_test.csv', index=False)

In [32]:
# Make predictions on the aligned data
prediction = model.predict(X_test_vh)

C:\Users\Hei Yee Lau\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [33]:
#test with the vh data
X_test_vh.loc[:, 'clm'] = prediction
X_test_vh.to_csv('result_vh.csv', index=False)